In [ ]:
import pandas as pd

games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")
ranking = pd.read_csv("Data/ranking.csv")
player = pd.read_csv("Data/players.csv")

Golden State Warriors우승 시즌과 이전 시즌의 스타팅 라인업 및 주요 선수 활용 변화 분석(감독관점)
======================================

## Title : 우승 시즌에 어떤 선수를 어떻게 썼길래 결과가 달라졌을까?

### 목적
**“GSW팀의 우승 시즌과 이전 시즌의 스타팅 라인업 및 주요 선수 활용 변화 분석하여,
향후 선수 발굴 및 영입에 활용한다.”**

**분석접근방식**
> 1.우승 시즌에 고정적으로 기용된 스타팅 멤버는 누구? 
> 2.이전 시즌 대비 출전 시간이 늘어난 선수는? 

**전처리 계획**
     1. 우승시즌과 직전 시즌 비교(우승시즌 2014-2015 -> 2015)


**사용할 컬럼** (games_details, games, teams)
1. 우승시즌비교
         SEASON
         GAME_ID
         TEAM_ID   


2. 스타팅 라인업 구성
         GAME_ID
         PLAYER_NAME == TEAM_ID로 해야함
         START_POSITION
         SEASON   

In [ ]:
games.info()

In [ ]:
details.info()

In [ ]:
teams.info()

In [ ]:
games.head(1)

In [ ]:
details.head(1)

In [ ]:
# 팀 약어 리스트
teams['ABBREVIATION'].unique()
# GSW

In [ ]:
#TEAM_ID
teams[teams['ABBREVIATION'] == 'GSW']

# TEAM_ID = 1610612744

In [ ]:
#사용할 컬럼 NULL값 확인하기
games[['GAME_ID', 'SEASON', 'HOME_TEAM_WINS']].isna().sum()

In [ ]:
#GAME_ID 중복 여부 확인 -> 0
games['GAME_ID'].duplicated().sum

In [ ]:
#games테이블에 SEASON 유니크값들
games['SEASON'].unique()

In [ ]:
#GSW가 참여한 경기 모두(홈경기 + 원정경기) => games_gsw 데이터프레임만듬
GSW_ID = 1610612744

games_gsw = games[(games['HOME_TEAM_ID'] == GSW_ID) |
                  (games['VISITOR_TEAM_ID'] == GSW_ID)]

games_gsw

In [ ]:
#GSW의 일부 필요 컬럼을 => games_gsw_part 데이터 프레임으로 만듬
games_gsw_part = games_gsw[['GAME_ID', 'SEASON', 'HOME_TEAM_WINS']]


In [ ]:
#details테이블에 위에만든 games_gsw_part 데이터 프레임의 GAME_ID 기준으로 INNER JOIN 진행
merged_gsw = details.merge(games_gsw_part, on='GAME_ID', how='inner')
merged_gsw

In [ ]:
#GSW만 필터링작업
gsw = merged_gsw = merged_gsw[merged_gsw['TEAM_ABBREVIATION'] == 'GSW']
gsw.head()

In [ ]:
#우승 직전/후로 나뉘니까 2016부터는 데이터 불필요.
gsw = gsw[gsw['SEASON'] <= 2015]

In [ ]:
#우승시즌 2014-2015, 우승직전시즌 전체
#champioship 2014-2015 우승시즌
#pre-championship 2013-2014 우승전 시즌
def season_group(season):
    if season == 2014:
        return "champioship(2014-2015)" 
    elif season == 2013:
        return "pre-championship(2013-2014)"
    
gsw['SEASON_GROUP'] = gsw['SEASON'].apply(season_group)

In [ ]:
#스타팅 라인업 -> START_POSITION 값이 있는 것만
starters = gsw[gsw['START_POSITION'].notna()]
starters

In [ ]:
#시즌별 스타팅 횟수 집계
starter_count = (starters.groupby(['SEASON_GROUP', 'PLAYER_NAME']).size().reset_index(name='start_games'))
starter_count.head(10)

In [ ]:
# 각 시즌별 스타팅 횟수가 많은 순서대로 나열.
starter_count.sort_values(['SEASON_GROUP', 'start_games'], ascending=False).head(10)

In [ ]:
# 각시즌 5명만 추리기
top_starters = (
    starter_count
    .sort_values(['SEASON_GROUP', 'start_games'], ascending=False)
    .groupby('SEASON_GROUP')
    .head(5)
)
top_starters

In [ ]:
!pip install matplotlib


In [ ]:
#pivot
plot_df = (
    top_starters
    .pivot(
        index='PLAYER_NAME',
        columns='SEASON_GROUP',
        values='start_games'
    )
)
plot_df

In [ ]:
!python -m pip install matplotlib


우승 이전 시즌 vs 우승 시즌의 스타팅 멤버 TOP5의 각 출전횟수를 비교
- 우승직전과 우승시즌의 핵심멤버가 누구인지 한눈에 보고 싶다.

In [ ]:
#막대그래프 시각화
import matplotlib.pyplot as plt

gsw_blue = '#1D428A'
gsw_yellow = '#FFC72C'

plot_df.plot(
    kind='bar',
    figsize=(10, 5),
    color=[gsw_blue, gsw_yellow]
)

plt.title("GSW Starting Lineup : Before vs Championship Season")
plt.xlabel("Player")
plt.ylabel("Number of Starts")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


그래프해석
- 주전선수가 바뀌었다. Draymond Green, Harrison Barnes선수는 새롭게 주전으로 출전했다.
- Draymond green은 직전시즌에서는 스타팅 출전이 거의 없었지만 우승시즌에 가장 많은 스타팅 멤버로 출전함
- Andre iguodala와 Daivid Lee는 이전 시즌에 스타팅 멤버로 출전했으나, 우승 시즌에는 스타팅 멤버로 출전이 거의 없다.
- Stephen Curry와 Klay Thompson은 우승 전후와 상관없이 높은 스타팅 출전 횟수를 유지하고있다.
- Andrew Bogut는 우승시즌에 스타팅 멤버 출전 횟수가 소폭 상승했다.

"우승 시즌에 고정적으로 기용된 스타팅 멤버는 누구"
Andrew Bogut, Draymond Green, Harrison Barnes, Klay Thompson, Stephen Curry 이렇게 5명이고 우승에 큰 기여를 했다고 보인다.

# **1/28(수) 진행 예정사항**

**2.이전 시즌 대비 출전 시간이 늘어난 선수는? (1/28 진행예정)**
스타팅횟수와 출전시간이 늘어났다면 그만큼 실력이 있어서 감독의 신뢰가 있는 선수가 아닐까?
우승시즌에 특정 선수가 진짜로 더 오래 뛰었을까?

**3.팀 성과 개선이 특정 선수때문일까? 라인업/운영 변화일까? (시간남을시)**


**전처리 계획**
     1. games_details.csv
          - MIN(TEXT) -> "32:12" TEXT형태를 분단위 실수로 변환
     2. gsw['SEASON_GROUP]


사용할 컬럼 (games_details, games, teams)
    1. MIN

In [ ]:
#MIN(출전시간) 데이터 셋 확인
# MIN                      str
details.dtypes

In [ ]:
#문자열 --> 숫자로 변환. 분단위의 실수로 계산
def min_to_float(x):
    if pd.isna(x):
        return 0

    x = str(x)

    if ':' in x:                         # "32:11"같은 형태
        m, s = x.split(':')
        return int(m) + int(s) / 60

    if x.isdigit():                      # "12" 같은 형태
        return int(x)

    return 0
gsw['MIN_FLOAT'] = gsw['MIN'].apply(min_to_float)

In [ ]:
gsw['MIN_FLOAT']

In [ ]:
#소수점 2자리 반올림
gsw['MIN_FLOAT'] = gsw['MIN_FLOAT'].round(2)
gsw['MIN_FLOAT']

In [ ]:
#위 사용했던 시즌 그룹 사용
gsw['SEASON_GROUP'] = gsw['SEASON'].apply(season_group)


In [ ]:
#선수별 출전시간? 경기수가 각각 다르니까 평균으로 계산
avg_minute = (gsw.groupby(['SEASON_GROUP', 'PLAYER_NAME'])['MIN_FLOAT'].mean().reset_index())
avg_minute['MIN_FLOAT'] = avg_minute['MIN_FLOAT'].round(2)
avg_minute.head()

In [ ]:
#top선수만 
top_minutes = (
    avg_minute
    .sort_values(['SEASON_GROUP', 'MIN_FLOAT'], ascending=False)
    .groupby('SEASON_GROUP')
    .head(5)
)
top_minutes

In [ ]:
#pivot
plot_df2 = (
    top_minutes
    .pivot(
        index='PLAYER_NAME',
        columns='SEASON_GROUP',
        values='MIN_FLOAT'
    )
)
plot_df2

GSW 우승이전시즌 VS우승시즌 평균 출전시간이 제일 높은 선수 비교
- 평균 출전시간을 선수별로 알고싶다.

In [ ]:
#막대그래프 시각화

plot_df2.plot(
    kind='bar',
    figsize=(10, 5),
    color=[gsw_blue, gsw_yellow]
)

plt.title("GSW TOP5 Players Avg Minutes| pre-championshipn vs championship")
plt.xlabel("Player")
plt.ylabel("Average Minutes")
plt.xticks(rotation=45)
plt.legend(title='Season')
plt.tight_layout()
plt.show()

그래프 해석
- Draymond Green 선수는 이전 시즌에 출전시간이 거의 없는 것으로 보아, 새로 영입된 선수이거나 그동안 기회가 없던 선수로 보인다.
- Draymond Green은 우승 시즌에는 약 30분 이상 출전했다.
- Daivid Lee는 이전시즌에서는 출전시간이 높았지만, 우승 시즌에서는 거의 비중이 급감한 것으로 보인다.
- Harrison Barnes선수는 우승 시즌에 평균 출전 시간이 증가했다.
- Klay Thompson, Stephen Curry선수는 우승시즌에 오히려 이전 시즌보다 평균 출전시간이 줄었다.
- Andre Igudala선수는 우승 전후로 평균 출전시간이 소폭 감소하였다.

**이전 시즌 대비 출전 시간이 늘어난 선수는?
Draymond Green, Harrison Barnes선수

# 최종결론 

### Top topic1 : 우승 시즌에 어떤 선수를 어떻게 썼길래 결과가 달라졌을까?
우승 시즌의 GSW팀은 팀의 기둥인 Stephen Curry와 Klay Thompson 두명의 선수를 여전히 주측으로 활용하면서도, 다른 라인업 변화를 시도 한 것으로 보입니다. 가장 눈에 띄는 변화는 Draymond Green선수 출전 입니다. 이전 시즌애 비해 Draymond Green선수의 선발 횟수와 출전 시간을 대폭 늘린반면, 기존 Daivid Lee와 Andre Iguodala의 선수의 출전 비중을 과감히 줄인 것으로 보입니다. 또한 Harrison Barnes 선수는 평균 출전 시간은 이전과 비슷하거나 늘어났음에도 불구하고, 오히려 선발 출전 횟수는 이전보다 줄어든 모습을 보입니다. 이는 Harrison Barnes선수를 무조건 선발로 고정하기보다, 경기 상황에 맞춰 Andre Iguodala 선수 같은 배테랑과 교체 투입하며 주전 선수들의 에너지 소모를 줄이는 유연한 로테이션 전술을 활용했다고 볼 수 있습니다.


### TO.감독관점 제안
Draymond Green 선수와 같은 포지션 핵심 인재를 영입하고 키워야합니다.

Curry,Thompson은 이전 시즌부터 우승 시즌까지 스타팅 멤버로 선발된 핵심 선수입니다. 이번 우승시즌에는 이전시즌과는 다르게 Draymond Green 선수의 출전시간을 증가시키고 두 선수의 출전 평균 시간이 오히려 소폭 감소였지만, 결과적으로 팀 성적이 상승했습니다.
이는 에이스에게만 의존하던 과거방식에서 벗어나, Draymond Green선수의 출전 시간을 늘려 에이스 선수들의 체력을 비축했기에 가능했던 결과입니다.

결과적으로 특정 선수를 과도하게 소모하기보다, 팀에 실질적인 도움을 줄 수 있는 조력자를 적재적소에 교체 투입하여 48분 내내 팀의 에너지 소모를 최소한으로 유지하는것이 우승을 향한 가장 확실한 전술이 될 것 입니다.